# Setup

## Verify we're in the Conda environment

In [ ]:
import sys
print(sys.executable)

## Import python packages

In [ ]:
import os
import sys
import json
import openai
from PIL import Image
import base64
import io
from dotenv import load_dotenv
import requests
from openai import OpenAI
import pprint
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

## openAI API key

In [ ]:
# Set up your OpenAI API key
# api_key = os.environ.get("OPENAI_API_KEY")

# Load the .env file
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


# Helper functions

## Function to base64 encode an image

In [ ]:
def encode_image(image_path):
    """Encode the image to base64 format to send to OpenAI."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

## Function to load existing results from JSON

In [ ]:
def load_existing_results(filename):
    """Load existing data from JSON file if it exists."""
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return []

## Function to review the generated stories for each image

In [ ]:
# Display for review
def review(input_file):
    inputs = []

    # Read the input file
    if os.path.exists(input_file):
        with open(input_file, 'r') as f:
            try:
                inputs = json.load(f)
            except json.JSONDecodeError:
                print(f"Error: Could not parse existing data in {input_file}.")
                sys.exit(1)
    else:
        print(f"Error: Could not find input_file {input_file}.")
        sys.exit(1)
    

    # Create a set of image paths
    inputs_image_paths = {entry["image"] for entry in inputs}

    # Base directory containing all the folders
    base_dir = Path("/Scandisk/onicai/charles/images")
    image_paths = [path for path in base_dir.glob("**/*.png") if not path.name.startswith(".")]
    thumbnail_size = (200, 200)

    icount = 0
    count_not_enough_sentences = 0
    for image_path in image_paths:
        icount += 1
        existing_entry = None

        # Find the image
        if str(image_path) in inputs_image_paths:
            existing_entry = next(entry for entry in inputs if entry["image"] == str(image_path))
            accepted_opening_sentences_with_stories = existing_entry["response"]["accepted_opening_sentences_with_stories"]
            # rejected_opening_sentences_with_stories = existing_entry["response"]["rejected_opening_sentences_with_stories"]
            if len(accepted_opening_sentences_with_stories) < 10:
                count_not_enough_sentences += 1
                print(f"------------------\n image {icount}: {image_path}")
                # Open and display the image
                image = Image.open(image_path)
                image.thumbnail(thumbnail_size)  # Resize the image to a thumbnail
                plt.figure(figsize=(4, 4))  # Adjust figure size
                plt.imshow(image)
                plt.axis('off')  # Hide axes for better view
                plt.show()
                print(f"# accepted opening sentences = {len(accepted_opening_sentences_with_stories)}")
                # print(json.dumps(existing_entry, indent=4))
            continue
        
        # Not yet processed, ERROR
        print("ERROR: Stories for this image were not yet judged.")
        sys.exit(1)

    print(f"Number of images with not exactly than 10 sentences = {count_not_enough_sentences}")

## Function to reformat the generated stories for each image

In [ ]:
def get_image_category_from_image_path(image_path):
    # Get the parent folder of the image
    parent_folder = image_path.parent

    # Find the subfolder name under 'images'
    category = None
    if 'images' in parent_folder.parts:
        category = parent_folder.parts[parent_folder.parts.index('images') + 1]
    else:
        sys.exit(f"ERROR: image_path is wrong: {image_path}")

    return category

def reformat(input_file, output_file, image_start, image_end):
    inputs = []
    outputs = []

    # Read the input file
    if os.path.exists(input_file):
        with open(input_file, 'r') as f:
            try:
                inputs = json.load(f)
            except json.JSONDecodeError:
                print(f"Error: Could not parse existing data in {input_file}.")
                sys.exit(1)
    else:
        print(f"Error: Could not find input_file {input_file}.")
        sys.exit(1)
    

    # Create a set of image paths
    inputs_image_paths = {entry["image"] for entry in inputs}

    # Base directory containing all the folders
    base_dir = Path("/Scandisk/onicai/charles/images")
    image_paths = [path for path in base_dir.glob("**/*.png") if not path.name.startswith(".")]
    image_paths = sorted(image_paths, key=lambda path: path.name)
    thumbnail_size = (200, 200)

    icount_images = 0
    icount_stories = 0
    count_not_exactly_10_sentences = 0
    # for image_path in image_paths:
    for image_index, image_path in enumerate(image_paths[image_start:image_end], start=image_start):
        # Process the image_path
        icount_images += 1
        existing_entry = None

        image_category = get_image_category_from_image_path(image_path)
        image_filename = image_path.name

        # Find the image
        if str(image_path) in inputs_image_paths:
            existing_entry = next(entry for entry in inputs if entry["image"] == str(image_path))
            accepted_opening_sentences_with_stories = existing_entry["response"]["accepted_opening_sentences_with_stories"]
            prompts = [ao["opening_sentence"] for ao in accepted_opening_sentences_with_stories]

            icount_stories += len(accepted_opening_sentences_with_stories)
            if len(accepted_opening_sentences_with_stories) != 10:
                count_not_exactly_10_sentences += 1

            # Open and display the image
            print(f"--------------------------------------------------")
            # image = Image.open(image_path)
            # image.thumbnail(thumbnail_size)  # Resize the image to a thumbnail
            # plt.figure(figsize=(2, 2))  # Adjust figure size
            # plt.imshow(image)
            # plt.axis('off')  # Hide axes for better view
            # plt.show()
            print(f"image_index     : {image_index}")
            print(f"image_category  : {image_category}")
            print(f"image_filename  : {image_filename}")
            print(f"image_path      : {str(image_path)}")
            print(f"# accepted opening sentences = {len(accepted_opening_sentences_with_stories)}")
            
            # reformat
            output = {
                "reviewed": False,
                "image_index": image_index,
                "image_category": image_category,
                "image_filename": image_filename,
                "image_path": str(image_path),
                "num_prompts": len(prompts),
                "prompts": prompts
            }
            outputs.append(output)

            # process next image_path
            continue
        
        # Not yet processed, ERROR
        print("ERROR: Stories for this image were not yet judged.")
        sys.exit(1)

    # Save the reformatted data to the output file
    with open(output_file, 'w') as f:
        json.dump(outputs, f, indent=4)

    print(f"Saved {icount_images} images with {icount_stories} total stories to : {output_file}")
    print(f"Number of images not with exactly 10 sentences = {count_not_exactly_10_sentences}")

# Run it

In [ ]:
input_file = './3-stories-judged-by-openai.json'
# review(input_file)

image_start = 0
image_end = 500

output_file = f'./5-reformat.json'
reformat(input_file, output_file, image_start, image_end)